In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mibitrans as mbt

# Formulation of the source depletion term

In the original BIOSCREEN, the source depletion term is expressed as; $C_0 \exp(-k_s(t - \frac{x}{v}))$. Although this represents both the decline in source concentration over time, and that concentrations at the advective front originate from the initial source concentration. However, this expression does not come forth from the exact solution, but is instead superimposed onto the analytical solution. When taking the exact solution of Karanovic (2007) as base, expression of source depletion is integrated into the longitudinal advection and dispersion term. This, mathematically sound, method of source depletion is implemented in the Anatrans model class.

Below, differences between the two implementations and the Mibitrans solution are visualized. Simultaneously, this example showcases how to adapt a model class to fit the desired behaviour.

In [ ]:
ft = 3.281
hydro = mbt.HydrologicalParameters(
    velocity=350/ft/365,     # [m/days]
    h_gradient=0.048,        # [-]
    h_conductivity=0.495,    # [m/day]
    porosity=0.25,           # [-]
    alpha_x=13.3/ft,         # [m]
    alpha_y=1.13/ft,         # [m]
    alpha_z=0                # T[m]
)

att = mbt.AttenuationParameters(
    bulk_density=1.7, #[g/m3]

    partition_coefficient=38, # [m^3/g]
    fraction_organic_carbon=5.7e-5, # [-]
    diffusion=0, # [m2/day]
    half_life=0 # [days]
)

source = mbt.SourceParameters(
    source_zone_boundary=np.array([30/ft, 35/ft, 40/ft]), # [m]
    source_zone_concentration=np.array([13.68, 2.508, 0.057]), # [g/m3]
    depth=10/ft, # [m]
    total_mass=25000 # [g]
)

model = mbt.ModelParameters(
    model_length = 1500/ft, # [m]
    model_width = 300/ft, # [m]
    model_time = 10 * 365, # [days]
    dx = 5/ft, # [m]
    dy = 2/ft, # [m]
    dt = 365/5 # [days]
)

To visualize the effect of the 'BIOSCREEN'-formulation of source depletion, swap out some equation elements from the Anatrans model class.

This is shown below. Make a custom class and initialize it with the same arguments. To inherrit the functionalities of Anatrans, assign it as the parent class by putting it between parentheses behind the class name. Then in the \_\_init\_\_, call the initialization of the parent class with super().\_\_init\_\_() and the class arguments. Then, the relevant portions of Anatrans can be adapted, while keeping all other functionalities the same.

In [ ]:
# Give the custom class a new name and make it inherit from Anatrans as a child class.
class AnatransAlternativeSource(mbt.Anatrans):
    """Mibitrans model class with alternative source depletion."""
    # Unless otherwise specified, all functionalities are now the same as Anatrans
    # Change the equation terms to fit the alternative source depletion formulation.
    def _equation_term_source_depletion(self, xxx, ttt):
        term = np.exp(-self.k_source * (ttt - xxx / self.rv))
        # Term can be max 1; can not have 'generation' of solute ahead of advection.
        return np.where(term > 1, 1, term)

    def _equation_decay_sqrt(self):
        return np.sqrt(1 + 4 * self._decay_rate * self._hyd_pars.alpha_x / self.rv)

In [ ]:
ana_object = mbt.Anatrans(hydro, att, source, model)
ana_results = ana_object.run()

alt_object = AnatransAlternativeSource(hydro, att, source, model)
alt_results = alt_object.run()

mbt_object = mbt.Mibitrans(hydro, att, source, model)
mbt_results = mbt_object.run()


There is a noticeable difference between the two implementations when the source mass is low (and therefore, $k_s$ is high). Over the model run time, these differences become more pronounced.

In [ ]:
time = 365 * 10
alt_results.centerline(time = time, color="red", linestyle="--", label="anatrans alternative")
ana_results.centerline(time = time, color="green", label="anatrans")
mbt_results.centerline(time = time, color="blue", linestyle=":", label="mibitrans")
plt.legend()
plt.show()

In [ ]:
time = 365 * 10
alt_results.transverse(time = time, x_position=300, color="red", linestyle="--", label="anatrans alternative")
ana_results.transverse(time = time, x_position=300, color="green", label="anatrans")
mbt_results.transverse(time = time, x_position=300, color="blue", linestyle=":", label="mibitrans")
plt.legend()
plt.show()

In [ ]:
def absolute_error(a, b):
    """Calculate the absolute error."""
    return b - a
def relative_error(a, b):
    """Calculate the relative error."""
    return (b - a) / a

Visualizing the relative error of both formulations of source depletion shows that for Anatrans, the error with Mibitrans gradually changes of the plume length. However, for the alternative, there is peak in error at the advective front. In front and behind the advection, the alternative method resolves to the regular Anatrans results.

In [ ]:
diff_ana = relative_error(mbt_results.cxyt[:,len(ana_results.y)//2,:],
                          ana_results.cxyt[:,len(ana_results.y)//2,:])
diff_alt = relative_error(mbt_results.cxyt[:,len(ana_results.y)//2,:],
                          alt_results.cxyt[:,len(ana_results.y)//2,:])
y_plottable = [diff_ana[9,:], diff_ana[29,:], diff_ana[49,:],
               diff_alt[9,:], diff_alt[29,:], diff_alt[49,:]]
color = ["blue", "green", "red", "lightblue", "lightgreen", "tomato"]
linestyle = ["-", "-", "-", "--", "--", "--"]
label = ["anatrans, t=2y", "anatrans, t=6y", "anatrans, t=8y",
         "alternative, t=2y", "alternative, t=6y", "alternative, t=8y"]

for i in range(len(y_plottable)):
    plt.plot(ana_results.x,
             y_plottable[i],
             color=color[i],
             linestyle=linestyle[i],
             label=label[i]
             )
plt.axhline(y=0, color='black', label="mibitrans")
plt.legend()
plt.xlabel("x-position [m]")
plt.ylabel("Relative error")
plt.title("Relative error with Mibitrans along plume centerline")
plt.show()

In [ ]:
lin_att = mbt.AttenuationParameters(
    bulk_density=1.7, #[g/m3]
    partition_coefficient=38, # [m^3/g]
    fraction_organic_carbon=5.7e-5, # [-]
    diffusion=0, # [m2/day]
    half_life=2 * 365 # [days]
)

ana_lin_object = mbt.Anatrans(hydro, lin_att, source, model)
ana_lin_results = ana_lin_object.run()

alt_lin_object = AnatransAlternativeSource(hydro, lin_att, source, model)
alt_lin_results = alt_lin_object.run()

mbt_lin_object = mbt.Mibitrans(hydro, lin_att, source, model)
mbt_lin_results = mbt_lin_object.run()

When also modelling linear decay, the plumes and their differences show similar behavior as before.

In [ ]:
time = 365 * 5
ana_lin_results.centerline(time = time, color="green", label="anatrans")
alt_lin_results.centerline(time = time, color="red", linestyle="--", label="anatrans alternative")
mbt_lin_results.centerline(time = time, color="blue", linestyle=":", label="mibitrans")
plt.legend()
plt.show()

In [ ]:
diff_ana = relative_error(mbt_lin_results.cxyt[:,len(ana_results.y)//2,:],
                          ana_lin_results.cxyt[:,len(ana_results.y)//2,:])
diff_alt = relative_error(mbt_lin_results.cxyt[:,len(ana_results.y)//2,:],
                          alt_lin_results.cxyt[:,len(ana_results.y)//2,:])
y_plottable = [diff_ana[9,:], diff_ana[29,:], diff_ana[49,:],
               diff_alt[9,:], diff_alt[29,:], diff_alt[49,:]]
color = ["blue", "green", "red", "lightblue", "lightgreen", "tomato"]
linestyle = ["-", "-", "-", "--", "--", "--"]
label = ["anatrans, t=2y", "anatrans, t=6y", "anatrans, t=8y",
         "alternative, t=2y", "alternative, t=6y", "alternative, t=8y"]

for i in range(len(y_plottable)):
    plt.plot(ana_results.x,
             y_plottable[i],
             color=color[i],
             linestyle=linestyle[i],
             label=label[i]
             )

plt.axhline(y=0, color='black', label="mibitrans")
plt.legend()
plt.xlabel("x-position [m]")
plt.ylabel("Relative error")
plt.title("Relative error with Mibitrans along plume centerline")
plt.show()

In [ ]:
large_source = mbt.SourceParameters(
    source_zone_boundary=np.array([30/ft, 35/ft, 40/ft]), # [m]
    source_zone_concentration=np.array([13.68, 2.508, 0.057]), # [g/m3]
    depth=10/ft, # [m]
    total_mass=500000 # [g]
)

ana_large_object = mbt.Anatrans(hydro, att, large_source, model)
ana_large_results = ana_large_object.run()

alt_large_object = AnatransAlternativeSource(hydro, att, large_source, model)
alt_large_results = alt_large_object.run()

mbt_large_object = mbt.Mibitrans(hydro, att, large_source, model)
mbt_large_results = mbt_large_object.run()


When increasing source mass, differences between two source depletion formulations become less apparent. Arguably, the alternative formulation comes closer to the exact Mibitrans solution than Anatrans does.

In [ ]:
time = 365*10
ana_large_results.centerline(time = time, color="green", label="anatrans")
alt_large_results.centerline(time = time, color="red", linestyle="--", label="anatrans alternative")
mbt_large_results.centerline(time = time, color="blue", linestyle=":", label="mibitrans")
plt.legend()
plt.show()

In [ ]:
time = 365*10
ana_large_results.transverse(time = time, x_position=150, color="green", label="anatrans")
alt_large_results.transverse(time = time, x_position=150, color="red", linestyle="--", label="anatrans alternative")
mbt_large_results.transverse(time = time, x_position=150, color="blue", linestyle=":", label="mibitrans")
plt.legend()
plt.show()

In [ ]:
# The absolute error does a better job visualizing the differences here
diff_ana = absolute_error(mbt_large_results.cxyt[:,len(ana_results.y)//2,:],
                          ana_large_results.cxyt[:,len(ana_results.y)//2,:])
diff_alt = absolute_error(mbt_large_results.cxyt[:,len(ana_results.y)//2,:],
                          alt_large_results.cxyt[:,len(ana_results.y)//2,:])
y_plottable = [diff_ana[9,:], diff_ana[29,:], diff_ana[49,:],
               diff_alt[9,:], diff_alt[29,:], diff_alt[49,:]]
color = ["blue", "green", "red", "lightblue", "lightgreen", "tomato"]
linestyle = ["-", "-", "-", "--", "--", "--"]
label = ["anatrans, t=2y", "anatrans, t=6y", "anatrans, t=8y",
         "alternative, t=2y", "alternative, t=6y", "alternative, t=8y"]

for i in range(len(y_plottable)):
    plt.plot(ana_results.x,
             y_plottable[i],
             color=color[i],
             linestyle=linestyle[i],
             label=label[i]
             )
plt.axhline(y=0, color='black', label="mibitrans")
plt.legend()
plt.xlabel("x-position [m]")
plt.ylabel("Absolute error [g/m3]")
plt.title("Absolute error with Mibitrans along plume centerline")
plt.show()